In [103]:
import numpy as np
import pandas as pd

In [104]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [105]:
df = pd.read_csv('titanic.csv')

In [106]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Creating A Plan To Implement PipeLine

In [107]:
df.drop(['PassengerId','Name','Ticket',"Cabin"], axis=1, inplace=True)

In [108]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['Survived'], axis=1),
    df['Survived'], 
    test_size=0.2,
    random_state=42
)

In [109]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [110]:
# Imputation Transformer
trf1 = ColumnTransformer(
    [('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'), [6])],
    remainder='passthrough'
)

In [111]:
# OneHotEncoding Transformer
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,6])
],remainder='passthrough')

In [112]:
# Scaling Transformer
trf3 = ColumnTransformer([
    ('scaling', MinMaxScaler(), slice(0,10))
])

In [113]:
# Feature Selection
trf4 = SelectKBest(score_func=chi2, k=8)

In [114]:
# Decision Tree Model
trf5 = DecisionTreeClassifier()

# Applying PipeLine

In [115]:
## With Make PipeLine Class
# pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [118]:
pipe = Pipeline([
    ('Pipe_Block1', trf1),
    ('Pipe_Block2', trf2),
    ('Pipe_Block3', trf3),
    ('Pipe_Block4', trf4),
    ('Pipe_Block5', trf5)
])

In [119]:
# Passing our data to all the connected pipeblocks 
pipe.fit(X_train, y_train)

Pipeline(steps=[('Pipe_Block1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('Pipe_Block2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('Pipe_Block3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('Pipe_Block4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001C79155DF80>)),
                ('Pipe_Block5', DecisionTreeClassifier())])

In [120]:
pipe.named_steps['Pipe_Block1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [121]:
y_pred = pipe.predict(X_test)

In [122]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6256983240223464

# Cross Validation Using Pipeline

In [123]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [126]:
# gridsearchcv
params = {
    'Pipe_Block5__max_depth':[1,2,3,4,5,None]
}

In [127]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Pipe_Block1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('Pipe_Block2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('Pipe_Block3',
                                        ColumnTransformer(transformers=[('scaling',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('Pipe_Block4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001C79155DF80>)),
                                       ('Pipe_Block5',
                                        DecisionTreeClassifier())]),
             param_grid={'Pipe_Block5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [128]:
grid.best_score_

0.6391214419383433

In [129]:
grid.best_params_

{'Pipe_Block5__max_depth': 2}


# Exporting Our Model

In [132]:
import pickle

In [133]:
pickle.dump(pipe, open('pipe.pkl','wb'))